In [1]:
## modules
import numpy as np
import matplotlib
import ipympl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipywidgets as widgets
%matplotlib widget
#########
def lwburk_general(sarr, vmax, km, itype, ki, inc = 1):
    """ calculations linear MM equations for all three types of inhibition """
    sarr = np.array(sarr)       ## to make sure
    #### 
    if itype == 'comp':
        km = km * (1 + inc/ki)
        slope = vmax/km
        vi = (1/(sarr * slope)) + (1/vmax)
    elif itype == 'noncomp':
        vmax = vmax / (1 + inc/ki)
        slope = vmax/km
        vi = (1/(sarr * slope)) + (1/vmax)
    elif itype == 'uncomp':
        slope = vmax/km
        vmax = vmax / (1 + inc/ki)
        vi = (1/(sarr * slope)) + (1/vmax)
    else:
        slope = vmax/km
        vi = (1/(sarr * slope)) + (1/vmax)
    return vi

def posrange(x, y):
    y[y == np.inf] = 0
    pos = np.where(y >=0)[0]
    return x[pos], y[pos]
###
inh_dict = {
    'No inhibition': 'normal',
    'Competitive': 'comp',
    'Non-competitive': 'noncomp',
    'Uncompetitive': 'uncomp'}
#############################
############################
km0 = 200                       ## uM
vmax0 = 4000                    ## s-1
sarrlin = np.linspace(-500, 500, 200)
realsubst = np.logspace(-2,4, 30, base = 10)
ki0 = 2
##########
normv = lwburk_general(sarrlin, vmax0, km0, 'normal', 1)
realvel = lwburk_general(realsubst, vmax0, km0, 'normal', 1)
##
norms, normv = posrange(sarrlin, normv)
############
## test
testf = np.loadtxt('/home/kazantsev/sci/praktikum/jupyter/test/wbwc_list.txt')
############
#fig = plt.figure(1,figsize=(8.5, 7.0))
#ax = fig.add_subplot(111)
###
fig=plt.figure(1, figsize=(16, 6.8), constrained_layout=False)
gs = fig.add_gridspec(6, 7)
ax = fig.add_subplot(gs[0:, 0:4])
###
ax.set_xlabel( r'1/[S]',fontsize=15, labelpad=3.3)
ax.set_ylabel(r'$1/V_{0}$',fontsize=15, labelpad=3.3)
ax.set_xticks([0])
ax.set_yticks([0])
ax.tick_params(labelsize=12, pad=1.6)
# #####
ax.plot(1/norms, normv, ls = 'solid', lw = 1.6, color = 'k')

ax.scatter(1/realsubst, realvel, s = 9, color = 'k')
####
ax.axvline(0, ls = 'solid', lw = 1.5, color = 'dimgray')
ax.axhline(0, ls = 'solid', lw = 1.5, color = 'dimgray')
# ###
### km, vmax lines
ax.scatter(0, 1/vmax0, marker = 'o', facecolor = 'b', s = 35, zorder = 100)
ax.scatter(-(1/km0), 0, marker = 'o', facecolor = 'b', s = 35, zorder = 100)
##
ax.plot([-(1/km0),-(1/km0)] , [-2, 0], color = 'b', ls = 'dashed', lw = 1.2, zorder = 5)
ax.plot([0,0.6/km0] , [1/vmax0, 1/vmax0], color = 'b', ls = 'dashed', lw = 1.2, zorder = 5)
# ###############
###km, vmax text
props0 = dict(boxstyle='square', facecolor='whitesmoke', alpha=1.0)
ax.text(-(1/km0) -(1/km0)*0.18, -0.2/vmax0, s = r'$-\left(\frac{1}{K_{m}}\right)$', fontsize = 18, rotation = 0, color = 'blue', bbox = props0, zorder = 100)
ax.text(0.5/km0, 1/vmax0, s = r'$\frac{1}{V_{max}}$', fontsize = 18, rotation = 0, color = 'b', bbox = props0, zorder = 100)
####
ax.set_xlim(-(1.5/km0) -(1/km0)*0.2 , (1/km0))
ax.set_ylim(-0.62/vmax0, 2/vmax0)
#####################################
## pictures with schemes
img_name0 = 'normal'
norm_img = mpimg.imread('normal.png')
ax2 = fig.add_subplot(gs[0:, 4:])
ax2.imshow(norm_img)
ax2.axis('off')
# ########
# ####
plt.subplots_adjust(left=0.06,right=0.96,bottom=0.10,top=0.98,wspace=0.12,hspace=0.12)

itypeprev = 'normal'
###########################
## INTERACTIVE REACTION ###
###########################
def plot_sel_rates(itype, inh, ki):
    global itypeprev
    itype = inh_dict[itype]
    selv = lwburk_general(sarrlin, vmax0, km0, itype, ki, inc = inh)
    sels, selv = posrange(sarrlin, selv)
    ## remove the interactive line from the plot
    try:
        sel_curve.remove()
        kapp_pos.remove()
        kapp_line.remove()
        vapp_pos.remove()
        vapp_line.remove()
        ### text
        print(len(ax.texts()))
        #kapp_txt.remove()
        #vapp_txt.remove()
    except:
        pass
    while len(ax.lines) > 5:
        ax.lines[-1].remove()
    ######
    visprop = 'yes'
    pcolor = 'red'
    if itype == 'normal':
        pcolor = 'black'
        visprop = 'no'
        kapp = km0
        vapp = vmax0
    elif itype == 'comp':
        kapp = km0 * (1 + inh/ki)
        vapp = vmax0
    elif itype == 'noncomp':
        kapp = km0
        vapp = vmax0 / (1 + inh/ki)
    else:
        kapp = km0 / (1 + inh/ki)
        vapp = vmax0 / (1 + inh/ki)
    ###
    sel_curve = ax.plot(1/sels, selv, ls = 'solid', lw = 1.6, color = pcolor)
    #####
    if itype != 'normal':
        kapp_pos = ax.plot(-(1/kapp), 0, marker = 'o', color = 'red', ms = 10, zorder = 50)
        vmax_pos = ax.plot(0, 1/vapp, marker = 'o', color = 'red', ms = 10, zorder = 50)
        ##
        kapp_line = ax.plot([-(1/kapp),-(1/kapp)] , [-2, 0], color = 'r', ls = 'dashed', lw = 1.2, zorder = 1)
        vmax_line = ax.plot([0,0.9/km0] , [1/vapp, 1/vapp], color = 'r', ls = 'dashed', lw = 1.2, zorder = 1)
        ## text
        props1 = dict(boxstyle='square', facecolor='whitesmoke', alpha=1.0)
        if len(ax.texts) > 3:
            ax.texts[-2].set_x(-(1/kapp) -(1/kapp)*0.18)
            ax.texts[-1].set_y(1/vapp)
        else:
            kapp_txt = ax.text(-(1/kapp) -(1/kapp)*0.18, -0.5/vapp, s = r'$-\left(\frac{1}{K_{m}^{app}}\right)$', fontsize = 18, rotation = 0, color = 'r', bbox = props1, zorder = 100)
            vapp_txt = ax.text(0.82/kapp, 1/vapp, s = r'$\frac{1}{V_{max}^{app}}$', fontsize = 18, rotation = 0, color = 'r', bbox = props1, zorder = 100)
    ############
    ### load schemes
    if itype != itypeprev:
        new_img = mpimg.imread('%(itype)s.png' % vars())
        ax2.clear()
        ax2.imshow(new_img)
        ax2.axis('off')
    ###########
    itypeprev = itype
    ######################
    ######################
###################
#print(help(widgets.interact()))
style = {'description_width': 'initial', 'fontweight': 'bold'}
widgets.interact(plot_sel_rates, 
                 itype = widgets.RadioButtons(options=['No inhibition', 'Competitive', 'Non-competitive', 'Uncompetitive'], 
                                              layout={'width': 'max-content'}, value ='No inhibition', description = 'Inhibition type', disabled=False, style = style), 
                 inh = widgets.FloatSlider(min = 0, max = 10, step = 0.25, value = 0, description = '[I]', style = style), 
                 ki = widgets.FloatSlider(min = 10, max = 20, step = 0.5, value = 10, description = '$K_{i}$', style = style))
####


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(RadioButtons(description='Inhibition type', layout=Layout(width='max-content'), options=…

<function __main__.plot_sel_rates(itype, inh, ki)>